<a href="https://colab.research.google.com/github/vivirocha/Bootcamp_DataScience/blob/main/DesenvolvendoSolu%C3%A7%C3%B5esUtilizandoApacheSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bootcamp Cientista de Dados - IGTI **

Trabalho 1 - Módulo 2

*Desenvolvendo Soluções Utilizando Apache Spark*


**Enunciado** <br> 
Dados do mercado financeiro são interessantes e ricos: cada ação negociada na bolsa de valores tem um preço que varia a cada dia. Você foi contratado como cientista de dados de uma empresa de Wall Street para criar modelos preditivos que, a partir da variação diária do preço das ações, consigam subsidiar e melhorar decisões de compra e venda de ações. Você disse que, como todo bom cientista de dados, gostaria de explorar os dados para entender suas características antes de criar qualquer modelo preditivo. <br>

Os dados estão disponíveis em https://www.kaggle.com/camnugent/sandp500/ por meio do arquivo all_stocks_5yr.csv. O arquivo contém, para cada dia e ação do S&P 500 (lista de 500 maiores empresas americanas), os seguintes dados: <br> 
● Date - no formato yy-mm-dd <br>
● Open - Preço da ação na abertura do mercado no dia, em dólares. <br>
● High - Maior preço alcançado naquele dia. <br>
● Low - Menor preço alcançado naquele dia. <br>
● Close - Preço da ação no fechamento do mercado no dia. <br>
● Volume - Número de ações vendidas / compradas.<br>
● Name - O nome da ação. <br>

Apesar do volume de dados ser pequeno, você decidiu usar o Apache Spark para processar os dados para aprender a ferramenta, e tendo em vista que a sua empresa disse que, em breve, obterá dados por minuto, e não por dia, e de todas as ações do planeta, não apenas dos Estados Unidos. Neste caso, uma ferramenta desenhada para lidar com big data será necessária, e você já quer estar com o código pronto.


In [374]:
#Instalando spark
!pip install spark

In [375]:
#Instalando pyspark
!pip install pyspark

In [376]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [377]:
#Importando biblioteca pandas
import pandas as pd

In [378]:
#Lendo o arquivo .csv e criando um dataframe dele.
df = spark.read.csv('all_stocks_5yr.csv', sep=',', header=True)

Para visualizarmos os dados do nosso dataframe, utilizaremos o comando **.show()**.

In [379]:
df.show()

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 12.7|13.26| 9419000| AAL|
|2013-02-27|13.28|13.62|13.18|13.41| 7390500| AAL|
|2013-02-28|13.49|13.63|13.39|13.43| 6143600| AAL|
|2013-03-01|13.37|13.95|13.32|13.61| 7376800| AAL|
|2013-03-04| 13.5|14.07|13.47| 13.9| 8174800| AAL|
|2013-03-05|14.01|14.05|13.71|1

Iremos renomear as colunas.

In [380]:
#Importando col a partir do pyspark.sql.functions
from pyspark.sql.functions import col

In [381]:
df = df.select(col("date").alias("data"), col("open").alias("abertura"), col("high").alias('maxima'), col('low').alias('minima'), col('close').alias('fechamento'), col('volume').alias('volume'), col('Name').alias('nome'))
df.show()

+----------+--------+------+------+----------+--------+----+
|      data|abertura|maxima|minima|fechamento|  volume|nome|
+----------+--------+------+------+----------+--------+----+
|2013-02-08|   15.07| 15.12| 14.63|     14.75| 8407500| AAL|
|2013-02-11|   14.89| 15.01| 14.26|     14.46| 8882000| AAL|
|2013-02-12|   14.45| 14.51|  14.1|     14.27| 8126000| AAL|
|2013-02-13|    14.3| 14.94| 14.25|     14.66|10259500| AAL|
|2013-02-14|   14.94| 14.96| 13.16|     13.99|31879900| AAL|
|2013-02-15|   13.93| 14.61| 13.93|      14.5|15628000| AAL|
|2013-02-19|   14.33| 14.56| 14.08|     14.26|11354400| AAL|
|2013-02-20|   14.17| 14.26| 13.15|     13.33|14725200| AAL|
|2013-02-21|   13.62| 13.95|  12.9|     13.37|11922100| AAL|
|2013-02-22|   13.57|  13.6| 13.21|     13.57| 6071400| AAL|
|2013-02-25|    13.6| 13.76|  13.0|     13.02| 7186400| AAL|
|2013-02-26|   13.14| 13.42|  12.7|     13.26| 9419000| AAL|
|2013-02-27|   13.28| 13.62| 13.18|     13.41| 7390500| AAL|
|2013-02-28|   13.49| 13

Limitaremos a visualização de apenas 5 observações.

In [382]:
df.limit(10).toPandas()

,data,abertura,maxima,minima,fechamento,volume,nome
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.1,14.27,8126000,AAL
3,2013-02-13,14.3,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
5,2013-02-15,13.93,14.61,13.93,14.5,15628000,AAL
6,2013-02-19,14.33,14.56,14.08,14.26,11354400,AAL
7,2013-02-20,14.17,14.26,13.15,13.33,14725200,AAL
8,2013-02-21,13.62,13.95,12.9,13.37,11922100,AAL
9,2013-02-22,13.57,13.6,13.21,13.57,6071400,AAL


Iremos alterar a posição das colunas com o comando **.select()**.

In [383]:
df = df.select('nome', 'data', 'abertura', 'maxima', 'minima', 'fechamento', 'volume')
df.show()

+----+----------+--------+------+------+----------+--------+
|nome|      data|abertura|maxima|minima|fechamento|  volume|
+----+----------+--------+------+------+----------+--------+
| AAL|2013-02-08|   15.07| 15.12| 14.63|     14.75| 8407500|
| AAL|2013-02-11|   14.89| 15.01| 14.26|     14.46| 8882000|
| AAL|2013-02-12|   14.45| 14.51|  14.1|     14.27| 8126000|
| AAL|2013-02-13|    14.3| 14.94| 14.25|     14.66|10259500|
| AAL|2013-02-14|   14.94| 14.96| 13.16|     13.99|31879900|
| AAL|2013-02-15|   13.93| 14.61| 13.93|      14.5|15628000|
| AAL|2013-02-19|   14.33| 14.56| 14.08|     14.26|11354400|
| AAL|2013-02-20|   14.17| 14.26| 13.15|     13.33|14725200|
| AAL|2013-02-21|   13.62| 13.95|  12.9|     13.37|11922100|
| AAL|2013-02-22|   13.57|  13.6| 13.21|     13.57| 6071400|
| AAL|2013-02-25|    13.6| 13.76|  13.0|     13.02| 7186400|
| AAL|2013-02-26|   13.14| 13.42|  12.7|     13.26| 9419000|
| AAL|2013-02-27|   13.28| 13.62| 13.18|     13.41| 7390500|
| AAL|2013-02-28|   13.4

Quantos registros há na planilha?

In [384]:
df.describe().show()


+-------+------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|  nome|      data|         abertura|           maxima|           minima|       fechamento|           volume|
+-------+------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|619040|    619040|           619029|           619032|           619032|           619040|           619040|
|   mean|  null|      null|83.02333431454696|83.77831069346465|82.25609641375786|83.04376276476573|4321823.395568945|
| stddev|  null|      null| 97.3787690433231|  98.207518904464| 96.5074210580914|97.38974800165752|8693609.511967659|
|    min|     A|2013-02-08|             1.62|             1.69|              1.5|             1.59|                0|
|    max|   ZTS|2018-02-07|           999.47|           999.75|           999.48|            999.6|           999994|
+-------+------+----------+-----------------+-----------

In [385]:
df.count()

619040

Quantos registros há na planilha para a ação da Apple (AAPL)?
<br> 


Para selecionarmos dados específicos de uma coluna específica utilizaremos as funções em conjunto: **select, where, count**. <br>
Então selecionaremos a coluna **nome**, onde exista nomes **igual a AAPL** *(ação da APPLE)* e **contaremos** quantas são.

In [386]:
df.select('nome').where(df.nome=="AAPL").count()

1259

Quantas empresas distintas têm registros nessa planilha?


In [387]:
#importando biblioteca para contagem de valores distintos 
from pyspark.sql.functions import countDistinct

Criaremos uma variável para inserir os valores distintos da coluna **nome** e visualizaremos com o comando **.show()**




In [388]:
empresasdistintas = df.select(countDistinct("nome"))
empresasdistintas.show()

+--------------------+
|count(DISTINCT nome)|
+--------------------+
|                 505|
+--------------------+



Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

Criaremos uma coluna para verificarmos em quais observações o preço de fechamento é maior do que o preço da abertura.


In [389]:
df = df.withColumn("fech_abert", col('fechamento') >col('abertura'))
df.limit(5).toPandas()

,nome,data,abertura,maxima,minima,fechamento,volume,fech_abert
0,AAL,2013-02-08,15.07,15.12,14.63,14.75,8407500,False
1,AAL,2013-02-11,14.89,15.01,14.26,14.46,8882000,False
2,AAL,2013-02-12,14.45,14.51,14.1,14.27,8126000,False
3,AAL,2013-02-13,14.3,14.94,14.25,14.66,10259500,True
4,AAL,2013-02-14,14.94,14.96,13.16,13.99,31879900,False


Vamos usar a função count para contarmos quantas vezes o fechamento foi maior que a abertura.

In [390]:
df.select('fech_abert').where(df.fech_abert=="True").count()


318940

Então, dividiremos o resultado pelo total dos eventos.

In [391]:
porcentagem =  (318940/619040)*100
round(porcentagem, 2)

51.52

Qual o maior valor das ações da Apple (AAPL) na história?


In [392]:
#Selecionando e agrupando as colunas de nome e maxima geral.
df.groupby('nome').agg({'maxima': 'max'}).show()

+----+-----------+
|nome|max(maxima)|
+----+-----------+
|   A|       75.0|
| AAL|      59.08|
| AAP|      99.99|
|AAPL|      99.99|
|ABBV|      99.49|
| ABC|      99.92|
| ABT|       64.6|
| ACN|      99.94|
|ADBE|      99.99|
| ADI|      98.38|
| ADM|      53.91|
| ADP|      99.88|
| ADS|      312.0|
|ADSK|      98.05|
| AEE|      64.89|
| AEP|      78.07|
| AES|       9.97|
| AET|      99.92|
| AFL|      91.73|
| AGN|      99.55|
+----+-----------+
only showing top 20 rows



In [393]:
#Filtrando pelo valor que desejamos encontrar.
apple = df[df['nome']=='AAPL']

In [394]:
#Agrupando por nome e procurando a maxima do valor que desejamos encontrar.
apple.groupby('nome').agg({'maxima': 'max'}).show()

+----+-----------+
|nome|max(maxima)|
+----+-----------+
|AAPL|      99.99|
+----+-----------+



Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

In [395]:
df.describe().show()

+-------+------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|  nome|      data|         abertura|           maxima|           minima|       fechamento|           volume|
+-------+------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|619040|    619040|           619029|           619032|           619032|           619040|           619040|
|   mean|  null|      null|83.02333431454696|83.77831069346465|82.25609641375786|83.04376276476573|4321823.395568945|
| stddev|  null|      null| 97.3787690433231|  98.207518904464| 96.5074210580914|97.38974800165752|8693609.511967659|
|    min|     A|2013-02-08|             1.62|             1.69|              1.5|             1.59|                0|
|    max|   ZTS|2018-02-07|           999.47|           999.75|           999.48|            999.6|           999994|
+-------+------+----------+-----------------+-----------

In [396]:
df.groupby('nome').agg({'fechamento': 'max'}).show(5)

+----+---------------+
|nome|max(fechamento)|
+----+---------------+
|   A|          74.82|
| AAL|          58.47|
| AAP|          99.95|
|AAPL|          99.99|
|ABBV|          99.95|
+----+---------------+
only showing top 5 rows



In [397]:
df.agg({'fechamento': 'stddev'}).show()

+------------------+
|stddev(fechamento)|
+------------------+
| 97.38974800165752|
+------------------+



In [398]:
dp = df.groupby('nome').agg({'fechamento': 'stddev'}).show(5)

+----+------------------+
|nome|stddev(fechamento)|
+----+------------------+
|ALXN|29.230294072399754|
| GIS| 5.626922424396725|
|   K| 5.791233527106072|
| LEN| 7.164922272045884|
| AIV| 6.048727635117268|
+----+------------------+
only showing top 5 rows

